In [83]:
# Convnet Visualiser (VGG19)

import time
import numpy as np
from tqdm import tqdm
from keras.preprocessing.image import save_img
from keras import backend as K
import tensorflow as tf
from matplotlib.image import imsave
from tensorflow.python.framework.ops import disable_eager_execution

disable_eager_execution()

In [54]:
from keras.applications.vgg19 import VGG19
model = VGG19(weights='imagenet', include_top=False)
print('Model Loaded')
# model.summary()

Model Loaded


In [55]:
for layer in model.layers:
    if 'conv' not in layer.name:
        continue
    filters, biases = layer.get_weights()
    print(layer.name, layer.output.shape)

block1_conv1 (None, None, None, 64)
block1_conv2 (None, None, None, 64)
block2_conv1 (None, None, None, 128)
block2_conv2 (None, None, None, 128)
block3_conv1 (None, None, None, 256)
block3_conv2 (None, None, None, 256)
block3_conv3 (None, None, None, 256)
block3_conv4 (None, None, None, 256)
block4_conv1 (None, None, None, 512)
block4_conv2 (None, None, None, 512)
block4_conv3 (None, None, None, 512)
block4_conv4 (None, None, None, 512)
block5_conv1 (None, None, None, 512)
block5_conv2 (None, None, None, 512)
block5_conv3 (None, None, None, 512)
block5_conv4 (None, None, None, 512)


In [56]:
def normalize(x):
    return x / K.sqrt(K.mean(K.square(x))+1e-5)

def deprocess_img(x):
    x -= x.mean()
    x /= (x.std()+1e-5)
    x *= 0.1

    x += 0.5
    x = np.clip(x, 0, 1)

    x *= 255
    x = np.clip(x, 0, 255).astype('uint8')

    return x

In [90]:
layer_name = 'block3_conv4'
filter_index = 17

input_img = model.input
layer_dict = dict([(layer.name, layer) for layer in model.layers[1:]])

layer_output = layer_dict[layer_name].output
loss = K.mean(layer_output[:, :, :, filter_index])

grads = K.gradients(loss, input_img)[0]
grads = normalize(grads)

iterate = K.function([input_img], [loss, grads])

In [91]:
img_width = 128
img_height = 128
input_img_data = np.random.random((1, img_width, img_height, 3)) * 20 + 128

step = 1.
iterations = 20
for i in tqdm(range(iterations)):
    loss_value, grads_value = iterate([input_img_data])
    input_img_data += (grads_value*step)

    # print('Current loss value:', loss_value)

100%|██████████| 20/20 [00:00<00:00, 26.33it/s]


In [92]:
img = deprocess_img(input_img_data[0])
imsave('%s_filter_%d.png' % (layer_name, filter_index), img)
print('%s_filter_%d.png:Image Saved!!' % (layer_name, filter_index))

block3_conv4_filter_17.png:Image Saved!!
